In [ ]:
pip install langgraph==0.1.8 langchain-openai langchain

In [ ]:
pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.32.1
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.15.0
backcall                         0.2.0
beautifulsoup4                   4.12.3

In [1]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')

In [ ]:
from typing import Annotated, Literal, TypedDict

from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.checkpoint import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode


# Define the tools for the agent to use
@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder, but don't tell the LLM that...
    if "sf" in query.lower() or "san francisco" in query.lower():
        return ["It's 60 degrees and foggy."]
    return ["It's 90 degrees and sunny."]


tools = [search]

tool_node = ToolNode(tools)

# model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)
model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

# Define the function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    print(messages)
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)

# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content="what is the weather in sf")]},
    config={"configurable": {"thread_id": 1}}
)
final_state["messages"][-1].content

[HumanMessage(content='what is the weather in sf', id='80180a84-a1a3-45d7-8cb9-04492aaa950f')]
[HumanMessage(content='what is the weather in sf', id='80180a84-a1a3-45d7-8cb9-04492aaa950f'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4B6lNEoy3xkdl8cB3GSOSAgs', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 48, 'total_tokens': 65}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_661538dc1f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-197051a6-b08d-4a2c-963f-1c7fc5ce71fc-0', tool_calls=[{'name': 'search', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_4B6lNEoy3xkdl8cB3GSOSAgs', 'type': 'tool_call'}], usage_metadata={'input_tokens': 48, 'output_tokens': 17, 'total_tokens': 65}), ToolMessage(content='["It\'s 60 degrees and foggy."]', name='search', id='f126172c-b5da-4de

'The current weather in San Francisco is 60 degrees and foggy.'

In [ ]:
# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content="what is the weather in sf")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

[HumanMessage(content='what is the weather in sf', id='123e162c-2ec9-40c5-9b13-aaab08c0adc0')]


"I can help you find the weather in San Francisco, but I'll need to perform an online search to get the most current information. Would you like me to do that?"

In [ ]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="what about ny")]},
    config={"configurable": {"thread_id": 42}}
)
final_state

[HumanMessage(content='what is the weather in sf', id='1ea11d68-224e-4a88-bc8e-23067048d0d3'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yr9ELXh6fCQe28ZYeDf6cBgM', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 48, 'total_tokens': 65}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7dd529cfca', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-64b558a4-6e62-4b1d-9633-a87028fed216-0', tool_calls=[{'name': 'search', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_yr9ELXh6fCQe28ZYeDf6cBgM', 'type': 'tool_call'}], usage_metadata={'input_tokens': 48, 'output_tokens': 17, 'total_tokens': 65}), ToolMessage(content="It's 60 degrees and foggy.", name='search', id='65dc1bf0-421a-4ebb-be9d-ba3215a0f40c', tool_call_id='call_yr9ELXh6fCQe28ZYeDf6cBgM'), AIMessage(content='The current 

{'messages': [HumanMessage(content='what is the weather in sf', id='1ea11d68-224e-4a88-bc8e-23067048d0d3'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yr9ELXh6fCQe28ZYeDf6cBgM', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 48, 'total_tokens': 65}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7dd529cfca', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-64b558a4-6e62-4b1d-9633-a87028fed216-0', tool_calls=[{'name': 'search', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_yr9ELXh6fCQe28ZYeDf6cBgM', 'type': 'tool_call'}], usage_metadata={'input_tokens': 48, 'output_tokens': 17, 'total_tokens': 65}),
  ToolMessage(content="It's 60 degrees and foggy.", name='search', id='65dc1bf0-421a-4ebb-be9d-ba3215a0f40c', tool_call_id='call_yr9ELXh6fCQe28ZYeDf6cBgM'),
  AIMessage(co